In [6]:
import matplotlib.pyplot as plt
import sys
import torch
import numpy as np
import pathlib
path = pathlib.Path()
# Insert all modules a folder above
sys.path.insert(0, str(path.absolute().parent))
from ssd.config.defaults import cfg
from ssd.modeling.box_head.prior_box import PriorBox
from ssd.utils.box_utils import convert_locations_to_boxes




ModuleNotFoundError: No module named 'torch'

In [ ]:
config_path = "configs/train_waymo.yaml"
cfg.merge_from_file(config_path)
prior_box = PriorBox(cfg)

In [ ]:
def plot_bbox(box):
    cx, cy, w, h = box
    x1, y1 = cx + w/2, cy + h/2
    print(y1)
    x0, y0 = cx - w/2, cy - h/2
    print(y0)
    plt.plot(
        [x0, x0, x1, x1, x0],
        [y0, y1, y1, y0, y0]
    )

In [11]:
priors = prior_box()
print("Prior box shape:", priors.shape)
print("First prior example:", priors[5])
locations = torch.zeros_like(priors)[None]
priors_as_location = convert_locations_to_boxes(locations, priors,cfg.MODEL.CENTER_VARIANCE, cfg.MODEL.SIZE_VARIANCE)[0]

aspect_ratio_indices = [0, 1, 2, 3]
plt.ylim([-100, 720 + 100])
plt.xlim([-100, 540 + 100])
ax = plt.gca()

def get_num_boxes_in_fmap(idx):
    boxes_per_location = cfg.MODEL.PRIORS.BOXES_PER_LOCATION[idx]
    feature_map_size = cfg.MODEL.PRIORS.FEATURE_MAPS[idx]
    return int(boxes_per_location * np.prod(feature_map_size))

layer_to_visualize = 5
offset = sum([get_num_boxes_in_fmap(prev_layer) for prev_layer in range(layer_to_visualize)])
boxees_per_location = cfg.MODEL.PRIORS.BOXES_PER_LOCATION[layer_to_visualize]
indices_to_visualize = []
colors = []
available_colors = ["r", "g", "b", "y", "m", "b", "w"]
for idx in range(offset, offset + get_num_boxes_in_fmap(layer_to_visualize)):
    for aspect_ratio_idx in aspect_ratio_indices:
        if idx % boxees_per_location == aspect_ratio_idx:
            indices_to_visualize.append(idx)
            colors.append(available_colors[aspect_ratio_idx])

ax.add_artist(plt.Rectangle([0,0], 720, 540, fill = False))
for i, idx in enumerate(indices_to_visualize):
    prior = priors_as_location[idx]
    color = colors[i]
    plot_bbox(ax, prior, color)

print(prior)

NameError: name 'prior_box' is not defined